In [1]:
# set up Python environment: numpy for numerical routines, and matplotlib for plotting
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# display plots in this notebook
%matplotlib inline

# for store the results
import pickle
import gzip

# for include pycode
import sys
import os
sys.path.insert(0,'../pycode')

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
# The caffe module needs to be on the Python path;
#  we'll add it here explicitly.
caffe_root = '/opt/caffe/'  # this file should be run from {caffe_root}/examples (otherwise change this line)
sys.path.insert(0, caffe_root + 'python')

import caffe
from caffe import layers as L
# If you get "No module named _caffe", either you have not built pycaffe or you have the wrong path.

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import aestheticNet
from preprocess import utilities 

In [ ]:
import utilsData
from sklearn.metrics import roc_auc_score, accuracy_score

# Generacion de train y test
Primero creamos dos txt con la informacion del fichero y de su clase

In [3]:
data = pickle.load(gzip.open('../packages/AVA_info.pklz','rb',2))

In [4]:
data.loc[:,'id'] = data['id'].apply(str)
classes = np.array(data.sort_values(['id']).loc[:,'Class'])
ids = np.array(data.sort_values(['id']).loc[:,'id'])
aux_list =  np.array(['/home/frubio/AVA/AVADataset/{:}.jpg'.format(i) for i in ids])

In [5]:
np.random.seed(1000)
random_idx = np.random.randint(3, size=len(ids))

In [6]:
train_files = aux_list[random_idx <= 1]
train_classes = classes[random_idx <= 1]
# Class balanced
new_indexes = utilities.balance_class(train_classes)
train_classes = train_classes[new_indexes]
train_files = train_files[new_indexes]

test_files = aux_list[random_idx == 2]
test_classes = classes[random_idx == 2]

In [ ]:
df_train = pd.DataFrame(list(zip(train_files,train_classes)))
df_train.columns = ['image_filename', 'label']
df_train.to_csv('../../AVA/train_partition_finetuning.txt', sep=' ', header=None, index=None)

df_test = pd.DataFrame(list(zip(test_files,test_classes)))
df_test.columns = ['image_filename', 'label']
df_test.to_csv('../../AVA/test_partition_finetuning.txt', sep=' ', header=None, index=None)

# Aqui comienza la chicha

In [7]:
caffe.set_device(0)  # if we have multiple GPUs, pick the first one
#caffe.set_mode_cpu()

In [ ]:
weights = caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'

In [ ]:
niter = 100000  # number of iterations to train

# Reset style_solver as before.
style_solver_filename = aestheticNet.solver(aestheticNet.aest_net(train=True,caffe_aes=True))
style_solver = caffe.get_solver(style_solver_filename)
style_solver.net.copy_from(weights)

In [ ]:
print ('Running solvers for %d iterations...' % niter)
solvers = [('pretrained', style_solver)]
loss, acc, weights = aestheticNet.run_solvers(niter, solvers)
print ('Done.')

train_loss = loss['pretrained']
train_acc = acc['pretrained']
style_weights = weights['pretrained']

# Delete solvers to save memory.
del style_solver, solvers

In [ ]:
os.rename(weights['pretrained'], "/home/frubio/AVA/aesthetic_finetuning.caffemodel")

# Testing

In [9]:
model_def = aestheticNet.aest_net(train=False,caffe_aes=True)
model_weights = "/home/frubio/AVA/aesthetic_finetuning.caffemodel"

net = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)

In [10]:
# load the mean ImageNet image (as distributed with Caffe) for subtraction
mu = np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy')
mu = mu.mean(1).mean(1)  # average over pixels to obtain the mean (BGR) pixel values

In [11]:
# create transformer for the input called 'data'
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
transformer.set_mean('data', mu)            # subtract the dataset-mean value in each channel
transformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

In [12]:
num_images = test_files.shape[0]
output_prob = np.zeros(num_images)
batch_size = net.blobs['data'].data.shape[0]

num_forwards = np.ceil(num_images/batch_size).astype(int)

In [13]:
for i in range(0,num_forwards):
    images_step = i*batch_size
    
    if (i == num_forwards - 1):
        images_to_process = num_images - images_step
    else:
        images_to_process = batch_size
            
    for j in range(0,images_to_process):
        # these lines is for those images that have four dimensions
        checked_image = caffe.io.load_image(test_files[j+images_step])
        if (len(checked_image.shape)==4):
            checked_image = checked_image[0]
                
        net.blobs['data'].data[j] = transformer.preprocess('data',checked_image)
        
    net.forward()
    output_prob[images_step:images_step+images_to_process] = net.blobs['probs'].data[0:images_to_process,1]

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000


ValueError: could not broadcast input array from shape (50) into shape (5)

In [26]:
results = {}
results['balanced'] = utilsData.balanced_accuracy(test_classes, output_prob)
results['AUC'] = roc_auc_score(test_classes, output_prob)
results['accuracy'] = accuracy_score(test_classes, (output_prob >= 0.5).astype(int))

In [28]:
pickle.dump(results, gzip.open( "../results/CNN_finetuning_CaffeNet.pklz", "wb" ), 2)